### Pre-Processing

Loading files and preparing the tweets (pre-processing is applied)

In [ ]:
%run scripts/pre_processing.py
df = pd.read_csv('data/tweet_df_class.csv', index_col='Datetime').drop('Unnamed: 0', axis=1)
tweets_text = df.original_text
prep_tweets = tweets_text.apply(pre_processing)

Creating two datasets: one of "malaria tweets" and one of "not malaria tweets"

In [ ]:
df['preproc'] = prep_tweets
nodup_df = df.drop_duplicates(subset='original_text')
tmp_tweets = nodup_df[['preproc', 'class']].reset_index(drop=True)
cases_tweets = tmp_tweets[tmp_tweets['class']==1]
not_cases_tweets = tmp_tweets[tmp_tweets['class']==0]

Creating a series with the most frequently used tokens in "malaria tweets" and their frequencies
(bag of words is used)

In [ ]:
vocab, bow_matrix = bag_of_words(cases_tweets.preproc)
bow_df = pd.DataFrame(bow_matrix, columns = vocab)
common_words = bow_df.sum(axis=0).sort_values()[-286:]

Oversampling

In [ ]:
#reading the labeled common words dataframe
words_df = pd.read_csv('data/common_words.csv')

#creating a dictionary with key = part of sentence and value = dataframe of tokens related to that part of sentence
#a is adjective, n is noun, v is verb and r is "other"
words_split = {}
for i, words in words_df.groupby('part'):
    words.freq = words.freq/sum(words.freq) #you may change words.freq with the frequency of only test data tokens
    words_split.update({str(i) : words.drop('part', axis=1)})

#absolute frequencies of each category
ss = words_df.groupby('part').sum()

#creating a series of uncommon words (index) and their relative frequencies 
noise_words = bow_df.sum(axis=0).sort_values()[:-286] #here too you can use noises from only test data tokens
noise_words /= sum(noise_words)

#generating fake tweets from the oversampling function
fake_tweets = pd.DataFrame(np.array([oversampler(words_split, noise_words) for i in range(250)]).T, #change the 250 to choose how many
            columns = ['preproc'])
fake_tweets.preproc = fake_tweets.preproc.str.split()
fake_tweets.insert(1, 'class', 1)

In [24]:
#this is just to show how to concatenate everything 
training_tweets = pd.concat([tmp_tweets, fake_tweets]).reset_index(drop=True)